# 7.范数

机器学习中出现的非常频繁的问题：**过拟合**和**规则化（又名：正则化）**。

此篇整理文档主要用于理解常用的L0、L1、L2和核范数规则化，还有规则化项参数的选择问题。

监督学习的问题无非就是“minimize error while regularing you parameters”，也就是在正则化参数的同时最小化误差。   

**最小化误差**是为了让我们的模型拟合我们的训练数据，而**正则化参数**是防止我们的模型过分拟合我们的训练数据。   

因为参数太多，会导致我们的模型复杂度上升，容易过拟合，也就是我们的训练误差会很小。   
但训练误差小并不是我们的最终目标，我们的目标是希望模型的测试误差小，也就是你能够准确的预测新的样本。  
所以，我们需要保证模型“简单”的基础上最小化训练误差，这样得到的参数才具有好的泛化性能，而模型“简单”就是通过规则函数实现的。   

<p style='color:blue;'>注意，在之前整理的“5偏差和方差”中，简单的定义是需要结合现实来看的。<a href='./5偏差和方差.ipynb'>5偏差和方差</a></p>


另外，规则项的使用还可以约束我们的模型的特性。   
这样就可以将人对这个模型的先验知识融入到模型的学习当中，强行地让学习到的模型具有人想要的特性，例如稀疏、低轶、平滑等等。   
要知道，有时候人的先验是非常重要的。  
由于人和机器的交流目前没有和人和人交流的那么直接的方法，目前这个媒介只能由规则项来担当了。

还有几种角度来看规则化的。  
规划则符合**奥卡姆剃刀(Occam's razor)原理**。   
思想：   
在所有可能选择的模型中，我们应该选择能够很好地解释已知数据并且十分简单的模型。   

从贝叶斯估计的角度来看，规则化项对应于模型的先验概率。   

还有个说法就是，规则化是**结构风险最小化策略**的实现，是在经验风险上加一个正则化项（regularizer）或惩罚项（penalty term）。

一般来说，监督学习可以看做最小化下面的目标函数：
$$w^* = arg min_{w} \sum_{i} L(y_i, f(x_i; w)) + \lambda \omega (w)$$

其中，第一项$L(y_i, f(x_i; w))$衡量我们的模型（分类或者回归）对第$i$个样本的预测值$f(x_i; w)$和真实的标签$y_i$之前的误差。  
因为我们的模型是要拟合我们的训练样本，所以我们要求这一项最小，也就是要求我们的模型尽量的拟合我们的训练数据。   

但正如上面所言，我们不仅要保证训练误差最小，我们更希望我们的模型测试误差小，所以我们需要加上第二项，也就是对参数$w$的规则化函数$\omega(w)$去约束我们的模型尽量的简单。

机器学习的大部分带参模型都和这个不但形似，而且神似。   
其实大部分无非就是变换这两项而已。  

对于第一项Loss Function，如果是Square Loss，那就是最小二乘；  
如果是Hinge Loss，那就是著名的SVM了；   
如果是exp-Loss，那就是牛逼的Bossting了；   
如果是log-Loss，那就是Logistic Regression了；  
......    

不同的loss函数，具有不同的拟合特性，这个也得具体问题具体分析。   
但是，我们先不研究loss function的问题，我们把目光转向“规则项$\omega(w)$”。

规则化函数$\omega(w)$也有很多选择，一般是模型复杂度的单调递增函数，模型越复杂，规则化项就越大。   
比如，规则化项可是模型参数向量的范数。   
然而，不同的选择参数$w$的约束不同，取得的效果也不同。   

我们常见的都聚集在： 
- 零范数 L0
- 一范数 L1
- 二范数 L2
- 迹范数
- Frobenius范数
- 核范数

那么多范数，到底它们表达了什么意思？  具有什么能力？  什么时候才能用？ 什么时候需要用呢？  

## L0范数和L1范数
L0范数是指向量中非0的元素的个数。   
如果我们用L0范数来规则化一个参数矩阵W的话，就是希望W的大部分元素都是0。   
换句话说，让参数W是稀疏的。  

我们知道非零元素的零次方为1，但零的零次方，非零数开零次方都是什么鬼，很不好说明L0的意义，所以在通常情况下，大家都用的是：
$$||x_0||_0 = \# (i| x_i \ne {0})$$ 
表示向量$x$中的非零元素的个数。  

对于L0范数，其优化问题为：
$$min ||x_0||_0$$
$$s.t. Ax = b $$

在实际应用中，由于L0范数本身不容易有一个好的数学表示形式，给出上面问题的形式化表示是一个很难的问题，故被人认为是一个NP难问题。  
所以，在实际情况中，L0的最优问题会被放宽到L1或L2的最优化。


针对上面这句“L0范数让参数W是稀疏的”，看到“稀疏”二字，大家都应该从当下风风火火的“压缩感知”和“稀疏编码”中醒悟过来，原来“稀疏”就是通过L0范数实现的。   
但是，在大部分papers中，稀疏都是通过L1范数来实现的。   
-- L0和L1有着某种不寻常的关系，所以这里将L0范数和L1范数一起来讲了。  

- L1范数是什么？
- 它为什么可以实现稀疏？
- 为什么大家都用L1范数去实现稀疏，而不是L0范数呢？

### L1范数是什么？
L1范数是指向量中各个元素绝对值之和，也有个美称叫“稀疏规则算子”（Lasso Regularization）。
定义如下：
$$||x||_1 = \sum_{i} |x_i|$$
表示向量x中非零元素的绝对值之和。

L1范数有很多的名字，例如我们熟悉的曼哈顿距离、最小绝度误差等。  
使用L1范数可以度量两个向量之间的差异，如绝对误差和(Sum of Absolute Difference)：
$$SAD(x_1, x_2) = \sum_{i} |x_1i - x_2i|$$

对于L1范数，它的优化问题如下：
$$ min ||x||_1 $$
$$ s.t. Ax = b $$ 

由于L1范数的天然性质，对L1优化的解也被叫做稀疏规则算子。  
通过L1可以实现特征的稀疏，去掉一些没有信息的特征，例如在对用户的电影爱好做分类的时候，用户有100个特征，可以只有十几个特征是对分类有用的，大部分特征如身高体重等可能都是无用的，利用L1范数就可以过滤掉。


### 为什么L1范数会使权值稀疏？
有人可能会这样给你回答“它是L0范数的最优凸近似”。   
实际上，还存在一个更美的回答：  
任何的规则化算子，如果他在$W_i = 0 $的地方不可微，并且可以分解为一个“求和”的形式，那么这个规则化算子就可以实现稀疏。  

W的L1范数是绝对值，$|w|$在w=0处不可微，但这还是不够直观。  
这里因为我们需要和L2范数进行对比分析。

### 既然L0可以实现稀疏，为什么不用L0，而要用L1呢？  
- 因为L0范数很难优化求解（NP难问题）
- L1范数是L0范数的最优凸近似，而且它比L0范数要更容易优化求解。
![title](../images/l0_l1.png)

总结：  
**L1范数和L0范数可以实现稀疏，L1因具有比L0更好的优化求解特性被而广泛地应用。 **

### 为什么要让参数W稀疏？让参数W稀疏有什么好处呢？
#### 1）特征选择 Feature Selection
大家对稀疏规则化趋之若鹜的一个关键原因在于它能实现特征的自动选择。   

一般来说，$x_i$的大部分元素，也就是特征，都是和最终的输出$y_i$没有关系或者不提供任何信息的，在最小化目标函数的时候考虑$x_i$这些额外的特征，虽然可以获得更小的训练误差，但在预测新的样本时，这些没用的信息反而会被考虑，从而干扰了对正确的$y_i$的预测。  

稀疏规则化算子的引入就是为了完成特征自动选择的光荣使命，它会学习地去掉这些没有信息的特征，也就是把这些特征对应的权重设置为0。

#### 2）可解释性 Interpretability
另一个青睐于稀疏的理由是，模型更容易解释。  

例如，换某种病的概率是$y$，然后我们收集到的数据$x$是1000维的，也就是我们需要寻找这1000种因素到底是怎么影响患这种病的概率的。  

假设我们这个是个回归模型：  
$y = w_1 * x_1 + w_2 * x_2 + w_3 * x_3 + ... + w_1000 * x_1000 + b$  
当然了，为了让y限定在[0,1]的范围内，一般还得加个Logistic函数。  

通过学习，如果最后学习到的$w^*$旧只有很少的非零元素，例如只有5个非零的$w_i$，那么我们就有理由相信，这些对应的特征在患病分析上面提供的信息是巨大的，决策性的。  

也就是说，患不患这种病只和这5个因素有关，那医生就好分析多了。  
但如果1000个$w_i$都是非0，医生面对这1000种因素，就非常困扰了。

## L2范数
除了L1范数，还有一种更受欢迎的规则化函数是 -- L2范数。
$$||w||_2$$

它也不逊于L1范数，它有两个美称，在回归里面，有人把有它的回归叫“岭回归”（Ridge Regression），有人也叫它“权值衰减”（weight decay）。

### L2范数是什么？
L2范数是指向量各元素的平方和然后求平方根。  

L2范数是我们最常见最常用的范数，我们用的最多的度量距离欧式距离就是一种L2范数，它的定义如下：
$$||x||_2 = \sqrt { \sum_{i} x_i ^ 2}$$

对于L2范数，它的优化问题如下：
$$min||x||_2 $$ 
$$s.t. Ax = b $$ 

L2范数通常会被用来做优化目标函数的正则化项，防止模型为了迎合训练集而过于复杂造成的过拟合的情况， 从而提高泛化能力。 


### L2范数的作用？
在改善机器学习里面一个非常重要的问题：过拟合。


### 为什么L2范数可以防止过拟合？
我们让L2范数的规则项$||w||_2$$最小，可以使得W的每个元素都很小，都接近于0，但与L1范数不同，它不会让它等于0，而是接近于0，这里是有很大的区别的。 
而越小的参数说明模型越简单，越简单的模型则越不容易产生过拟合现象。  

通过L2范数，我们可以实现了对模型空间的限制，从而在一定程度上避免了过拟合。

### L2范数的好处是什么呢？
- 学习理论的角度：L2范数可以防止过拟合，提升模型的泛化能力
- 优化计算的角度：
    从优化或者数值计算的角度来说，L2范数有助于处理condition number不好的情况下矩阵求逆很困难的问题。
    > 优化有两大难题：一是局部最小值，二是ill-condition病态问题。
<a href= 'https://blog.csdn.net/zouxy09/article/details/24971995'>此处的参考阅读链接</a>

## 核范数
核范数$||w||_*$是指矩阵奇异值的和，Nuclear Norm。  
作用：  
    约束Low-Rank（低轶）
    
    <a href='https://blog.csdn.net/zouxy09/article/details/24972869'>参考博文</a>